# 🚀 SageMaker Data Processing Job for Tabular Data Cleanup

This notebook demonstrates how to use AWS SageMaker Processing Jobs to clean tabular data from a CSV file. The key tasks include:

- Removing highly correlated columns
- Removing rows with excessive missing data
- Imputing missing values
- Encoding categorical features

## 📚 Prerequisites
- An AWS account with SageMaker access
- SageMaker execution role with necessary permissions
- Input dataset in CSV format

In [1]:
# 📦 Step 1: Setup Environment
import sagemaker
import boto3
import pandas as pd
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

print(f"SageMaker Role: {role}")
print(f"Default Bucket: {bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
SageMaker Role: arn:aws:iam::485186561655:role/service-role/AmazonSageMaker-ExecutionRole-20241205T130405
Default Bucket: sagemaker-us-east-1-485186561655


## 📊 Step 2: Explore Sample Data
Upload your CSV dataset to the S3 bucket and explore it.

In [3]:
# Example: Load a sample dataset

df = pd.read_csv('kaggle_london_house_price_data_sampled_data.csv')  # Replace with your dataset path
print(df.head())

                                         fullAddress  postcode  country  \
0  Flat 27, Rossetti Garden Mansions, Flood Stree...   SW3 5QX  England   
1                  31 Gladstone Road, London, W4 5SX    W4 5SX  England   
2                    53 Bexley Road, London, SE9 2PE   SE9 2PE  England   
3  Flat 17 Long Island House, 44 Warple Way, Lond...    W3 0RG  England   
4             51 Norwood Park Road, London, SE27 9UB  SE27 9UB  England   

  outcode   latitude  longitude  bathrooms  bedrooms  floorAreaSqM  \
0     SW3  51.484998  -0.164776        2.0       3.0         138.0   
1      W4  51.497203  -0.264610        1.0       2.0          80.0   
2     SE9  51.452279   0.068871        1.0       5.0         220.0   
3      W3  51.504021  -0.255028        NaN       NaN          74.0   
4    SE27  51.425938  -0.095586        1.0       3.0          97.0   

   livingRooms  ... saleEstimate_upperPrice saleEstimate_confidenceLevel  \
0          2.0  ...               2553000.0         

## 📝 Step 3: Create a Data Processing Script
We'll define a Python script that will perform data cleanup tasks.

In [13]:
%%writefile preprocessing_script.py
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import sys

# Load dataset
df = pd.read_csv('/opt/ml/processing/input/data.csv')

# Remove unnecessary data 
df = df.drop(columns=['fullAddress', 'rentEstimate_lowerPrice', 'rentEstimate_currentPrice', 'rentEstimate_upperPrice', 
                      'saleEstimate_lowerPrice', 'saleEstimate_upperPrice', 'outcode', 'saleEstimate_ingestedAt'])

# Correct date fields and drop originals
df['saleEstimate_valueChange.saleDate'] = pd.to_datetime(df['saleEstimate_valueChange.saleDate'])
df['history_date'] = pd.to_datetime(df['history_date'])
#df = df.drop(columns=['saleEstimate_valueChange.saleDate', 'history_date'])

# Remove rows with excessive missing data
df = df.dropna(thresh=int(df.shape[1] * 0.7))

# Find numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns

# Impute missing values for numeric columns
num_imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])

# Target Encoding postcode and current price instead of One Host Encoding due to high cardinality
target_mean = df.groupby('postcode')['saleEstimate_currentPrice'].mean()
df['Postcode_Encoded'] = df['postcode'].map(target_mean)

# Drop original Postcode column as no longer needed
df = df.drop(columns=['postcode'])

# Encode categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_cols)

# Save processed data
df.to_csv('/opt/ml/processing/output/processed.csv', index=False)

Overwriting preprocessing_script.py


## ⚙️ Step 4: Upload Source Data to S3
Upload the source CSV dataset to input location in S3

In [5]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file('kaggle_london_house_price_data_sampled_data.csv', bucket, 'input/data.csv')

## ⚙️ Step 5: Run SageMaker Processing Job
Run the preprocessing script using a SageMaker Processing Job.

In [12]:
input_raw_data_prefix = "input/"
output_preprocessed_data_prefix = "output"

processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve('sklearn', 'us-east-1', '1.2-1'),
    role=role,
    command=['python3'],
    instance_type='ml.m5.4xlarge',
    instance_count=1
)

processor.run(
    code='preprocessing_script.py',
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "data.csv"),
                            destination='/opt/ml/processing/input')], 
    outputs=[ProcessingOutput(source='/opt/ml/processing/output',
                            destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "data-processed.csv"))]
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2025-01-13-00-25-49-668


...............
..

## 📥 Step 6: Validate Cleaned Data
Download the processed data from S3 and validate.

In [ ]:
df_cleaned = pd.read_csv(f's3://{bucket}/output/processed.csv')
print(df_cleaned.head())